In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [35]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB

db = DB()

In [36]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz']

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp"
})

In [46]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['pop', 'jazz'],
 ['hip-hop', 'pop', 'k-pop'],
 ['ballad'],
 ['ballad'],
 ['pop', 'ballad', 'hip-hop'],
 ['pop', 'hip-hop', 'ballad'],
 ['pop', 'jazz'],
 ['hip-hop'],
 ['hip-hop', 'jazz'],
 ['k-pop', 'hip-hop', 'ballad']]

In [54]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623c3b95a96fb402f17e7195', '623c3b96a96fb402f17e7196', '623c3b96a96fb402f17e7197', '623c3b96a96fb402f17e7198', '623c3b96a96fb402f17e7199', '623c3b96a96fb402f17e719a', '623c3b97a96fb402f17e719b', '623c3b97a96fb402f17e719c', '623c3b97a96fb402f17e719d', '623c3b97a96fb402f17e719e']


,id,name,artists,artists_name
0,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan
1,7tIOwGFRTM08JdAeKOELGp,Blue room,3rxeQlsv0Sc2nyYaZ5W71T,Chet Baker


In [62]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
print(genre_seed_list)
features_list[0]

[     count
pop      1
jazz     1,       count
k-pop     2
pop       1,       count
k-pop     1,       count
k-pop     1,       count
pop       1
k-pop     1
k-pop     1,       count
pop       1
k-pop     1
k-pop     1,      count
pop      1
jazz     1,       count
k-pop     1,       count
k-pop     1
jazz      1,       count
k-pop     2
k-pop     1]


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3gzIK2dGSVvPBiohsTwEWZ,0.639,0.5690,0,-7.236,0.0487,0.0581,0.000169,0.108,0.616,142.967
1,7tIOwGFRTM08JdAeKOELGp,0.632,0.0378,5,-20.596,0.0583,0.9890,0.935000,0.112,0.357,121.361


In [64]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

In [65]:
reco_list[0].head()

,id,name,artists,artists_name
0,3567eNX2L3RkkMkwspncMH,Lie To Me,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer
1,1ASqQDG45adRth1NuejqqG,What Is This Thing Called Love,0CrCKxXekxMpkYfMEf8mca,Rebecca Ferguson
2,7nJc3vqcU2em6FH0c3Geox,Birdland,162DCkd8aDKwvjBb74Gu8b,Weather Report
3,1tckg6GRaTodmxnDd4Cxkl,Cantaloupe Island,2ZvrvbQNrHKwjT7qfGFFUW,Herbie Hancock
4,14FkDSrkQPko4YMhx6dLI2,Chronology,47odibUtrN3lnWx0p0pk2P,Ornette Coleman


In [66]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3567eNX2L3RkkMkwspncMH,0.633,0.478,0,-5.571,0.0359,0.0115,0.0000,0.1030,0.692,159.951
1,1ASqQDG45adRth1NuejqqG,0.532,0.482,0,-7.944,0.0381,0.2220,0.0000,0.0696,0.575,172.604
2,7nJc3vqcU2em6FH0c3Geox,0.585,0.565,0,-12.139,0.0372,0.4840,0.0226,0.0808,0.577,156.208
3,1tckg6GRaTodmxnDd4Cxkl,0.485,0.551,0,-11.115,0.0283,0.3900,0.0000,0.0814,0.452,116.162
4,14FkDSrkQPko4YMhx6dLI2,0.576,0.422,0,-11.277,0.0594,0.1850,0.2440,0.2640,0.570,115.478


In [67]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3gzIK2dGSVvPBiohsTwEWZ,0.562162,0.631358,0.000000,0.826061,0.044661,0.057780,0.000172,0.088251,0.623417,0.697783
1,7tIOwGFRTM08JdAeKOELGp,0.552703,0.039749,0.454545,0.217572,0.063716,0.993966,0.952138,0.093567,0.343145,0.521771
2,3567eNX2L3RkkMkwspncMH,0.554054,0.530009,0.000000,0.901895,0.019254,0.010916,0.000000,0.081606,0.705660,0.836142
3,1ASqQDG45adRth1NuejqqG,0.417568,0.534464,0.000000,0.793815,0.023620,0.222611,0.000000,0.037214,0.579050,0.939219
4,7nJc3vqcU2em6FH0c3Geox,0.489189,0.626903,0.000000,0.602751,0.021834,0.486099,0.023014,0.052100,0.581214,0.805650


In [69]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

-- 라운드 10
1 round : ECV 47.49%
2 round : ECV 65.3%
3 round : ECV 66.86%
4 round : ECV 67.37%
5 round : ECV 67.65%
6 round : ECV 67.69%
7 round : ECV 67.71%
8 round : ECV 67.71%
9 round : ECV 67.71%
10 round : ECV 67.71%
11 round : ECV 67.71%
Clustering End.


In [70]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
0,3567eNX2L3RkkMkwspncMH,Lie To Me,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer
7,6SL8U8TtdwOtGhbmGzsMfX,I'm Not Mad,26VFTg2z8YR0cCuwLzESi2,Halsey
9,2ulQ2Rx5fi0R8c0M3ETpj7,My Girl,19OQfCaZNNYap4BkxUVh43,Jazz Jamaica
12,0NuWgxEp51CutD2pJoF4OM,blind,1dy5WNgIKQU6ezkpZs4y8z,ROLE MODEL
14,20L6erw0ny2hiwQg8c1Dj4,Daze Inn,3mPc8WGusz2XF3Tvs3AKCR,Carlie Hanson


In [71]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(623c3b95a96fb402f17e7195)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e719f)가 등록되었습니다.
우체통(623c3b96a96fb402f17e7196)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a0)가 등록되었습니다.
우체통(623c3b96a96fb402f17e7197)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a1)가 등록되었습니다.
우체통(623c3b96a96fb402f17e7198)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a2)가 등록되었습니다.
우체통(623c3b96a96fb402f17e7199)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a3)가 등록되었습니다.
우체통(623c3b96a96fb402f17e719a)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a4)가 등록되었습니다.
우체통(623c3b97a96fb402f17e719b)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a5)가 등록되었습니다.
우체통(623c3b97a96fb402f17e719c)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a6)가 등록되었습니다.
우체통(623c3b97a96fb402f17e719d)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a7)가 등록되었습니다.
우체통(623c3b97a96fb402f17e719e)에
추천 플레이리스트 MuLetter(623c4043a96fb402f17e71a8)가 등록되었습니다.
